# Assignment 2

In this assigment, we will work with the *Adult* data set. Please download the data from the [UCI Machine Learning Repository](https://archive.ics.uci.edu/dataset/2/adult). Extract the data files into the subdirectory: `../05_src/data/adult/` (relative to `./05_src/`).

# Load the data

Assuming that the files `adult.data` and `adult.test` are in `../05_src/data/adult/`, then you can use the code below to load them.

In [2]:
import pandas as pd
columns = [
    'age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status',
    'occupation', 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week',
    'native-country', 'income'
]
adult_dt = (pd.read_csv('C:/Users/sijia/OneDrive/_DSI@UofT/6_production/production/05_src/data/adult/adult.data', header = None, names = columns)
              .assign(income = lambda x: (x.income.str.strip() == '>50K')*1))


# Get X and Y

Create the features data frame and target data:

+ Create a dataframe `X` that holds the features (all columns that are not `income`).
+ Create a dataframe `Y` that holds the target data (`income`).
+ From `X` and `Y`, obtain the training and testing data sets:

    - Use a train-test split of 70-30%. 
    - Set the random state of the splitting function to 42.

In [3]:
from sklearn.model_selection import train_test_split

# Create the features dataframe `X` (all columns except 'income')
X = adult_dt.drop(columns = 'income', axis = 1)

# Create the target dataframe `Y` (only the 'income' column)
Y = adult_dt['income']

# Split the data into training and testing sets (70-30 split, random state=42)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=42)

## Random States

Please comment: 

+ What is the [random state](https://scikit-learn.org/stable/glossary.html#term-random_state) of the [splitting function](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html)? 
+ Why is it [useful](https://en.wikipedia.org/wiki/Reproducibility)?

> **1. What is the `random_state` of the `train_test_split` function?**
> - The `random_state` parameter in the `train_test_split` function is used to control the shuffling of data before splitting it into training and testing sets.
> - It acts as a seed for the random number generator. By setting a `random_state` value (like 42), you ensure that the splitting process produces the same result every time the code is run. This allows you to obtain a consistent split of your data, which is important for reproducibility.
> - If you don’t specify a `random_state`, the split will be different every time you run the code, as the data will be shuffled differently.

> **2. Why is `random_state` useful for reproducibility?**
> - **Reproducibility:** In data science and machine learning, reproducibility refers to the ability to consistently recreate an experiment or analysis under the same conditions. Using `random_state` makes your results reproducible.
> - **Collaboration:** When working with others, setting a `random_state` allows colleagues to obtain the exact same data splits, ensuring consistency across different environments and making collaboration more efficient.
> - **Debugging and Comparison:** When you set a `random_state`, you can debug your code more easily and compare the performance of different models or algorithms on the exact same split of the data. This helps in understanding whether improvements in performance are due to changes in the model or just variations in the data split.

# Preprocessing

Create a [Column Transformer](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html) that treats the features as follows:

- Numerical variables

    * Apply [KNN-based imputation for completing missing values](https://scikit-learn.org/stable/modules/generated/sklearn.impute.KNNImputer.html):
        
        + Consider the 7 nearest neighbours.
        + Weight each neighbour by the inverse of its distance, causing closer neigbours to have more influence than more distant ones.
    * [Scale features using statistics that are robust to outliers](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.RobustScaler.html#sklearn.preprocessing.RobustScaler).

- Categorical variables: 
    
    * Apply a [simple imputation strategy](https://scikit-learn.org/stable/modules/generated/sklearn.impute.SimpleImputer.html#sklearn.impute.SimpleImputer):

        + Use the most frequent value to complete missing values, also called the *mode*.

    * Apply [one-hot encoding](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html):
        
        + Handle unknown labels if they exist.
        + Drop one column for binary variables.
    
    
The column transformer should look like this:

![](./images/assignment_2__column_transformer.png)

In [11]:
adult_dt.dtypes

age                int64
workclass         object
fnlwgt             int64
education         object
education-num      int64
marital-status    object
occupation        object
relationship      object
race              object
sex               object
capital-gain       int64
capital-loss       int64
hours-per-week     int64
native-country    object
income             int32
dtype: object

In [15]:
# List numerical features
numerical_features = adult_dt.select_dtypes(include=['int64', 'float64']).columns.tolist()
numerical_features

['age',
 'fnlwgt',
 'education-num',
 'capital-gain',
 'capital-loss',
 'hours-per-week']

In [16]:
# List categorical features
categorical_features = adult_dt.select_dtypes(include=['object', 'category']).columns.tolist()
categorical_features

['workclass',
 'education',
 'marital-status',
 'occupation',
 'relationship',
 'race',
 'sex',
 'native-country']

In [19]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.preprocessing import RobustScaler, OneHotEncoder

# Define the transformations for numerical variables
num_transforms = [
    ('knn_imputer', KNNImputer(n_neighbors=7, weights='distance')),
    ('scaler', RobustScaler())
]

# Define the transformations for categorical variables
cat_transforms = [
    ('simple_imputer', SimpleImputer(strategy='most_frequent')),
    ('one_hot_encoder', OneHotEncoder(handle_unknown='ignore', drop='if_binary'))
]

# Create the ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num_transforms', Pipeline(num_transforms), numerical_features),  
        ('cat_transforms', Pipeline(cat_transforms), categorical_features) 
    ]
)
preprocessor

ColumnTransformer(transformers=[('num_transforms',
                                 Pipeline(steps=[('knn_imputer',
                                                  KNNImputer(n_neighbors=7,
                                                             weights='distance')),
                                                 ('scaler', RobustScaler())]),
                                 ['age', 'fnlwgt', 'education-num',
                                  'capital-gain', 'capital-loss',
                                  'hours-per-week']),
                                ('cat_transforms',
                                 Pipeline(steps=[('simple_imputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('one_hot_encoder',
                                                  OneHotEncoder(drop='if_binary',
                                                                handle_unknown='ignore'))]),
                                 ['workclass', 'education', 'marital-status',
                                  'occupation', 'relationship', 'race', 'sex',
                                  'native-country'])])

## Model Pipeline

Create a [model pipeline](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html): 

+ Add a step labelled `preprocessing` and assign the Column Transformer from the previous section.
+ Add a step labelled `classifier` and assign a [`RandomForestClassifier()`](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html) to it.

The pipeline looks like this:

![](./images/assignment_2__pipeline.png)

In [21]:
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier

# Create the model pipeline
model_pipeline = Pipeline([
    ('preprocessing', preprocessor),  # The ColumnTransformer defined earlier
    ('classifier', RandomForestClassifier())  # The RandomForestClassifier
])

# Now, `model_pipeline` is a complete pipeline that can be used for fitting and predicting
model_pipeline

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('num_transforms',
                                                  Pipeline(steps=[('knn_imputer',
                                                                   KNNImputer(n_neighbors=7,
                                                                              weights='distance')),
                                                                  ('scaler',
                                                                   RobustScaler())]),
                                                  ['age', 'fnlwgt',
                                                   'education-num',
                                                   'capital-gain',
                                                   'capital-loss',
                                                   'hours-per-week']),
                                                 ('cat_transforms',
                                                  Pipeline(steps=[('simple_imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('one_hot_encoder',
                                                                   OneHotEncoder(drop='if_binary',
                                                                                 handle_unknown='ignore'))]),
                                                  ['workclass', 'education',
                                                   'marital-status',
                                                   'occupation', 'relationship',
                                                   'race', 'sex',
                                                   'native-country'])])),
                ('classifier', RandomForestClassifier())])

# Cross-Validation

Evaluate the model pipeline using [`cross_validate()`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_validate.html):

+ Measure the following [preformance metrics](https://scikit-learn.org/stable/modules/model_evaluation.html#common-cases-predefined-values): negative log loss, ROC AUC, accuracy, and balanced accuracy.
+ Report the training and validation results. 
+ Use five folds.


In [24]:
from sklearn.model_selection import cross_validate
from sklearn.metrics import roc_auc_score, log_loss, accuracy_score, balanced_accuracy_score

# Perform cross-validation
scoring = ['neg_log_loss', 'roc_auc', 'accuracy', 'balanced_accuracy']
cv_results = cross_validate(model_pipeline, X, Y, cv=5, scoring=scoring, return_train_score=True)
cv_results

c:\Users\sijia\miniconda3\envs\dsi_participant\lib\site-packages\sklearn\preprocessing\_encoders.py:242: UserWarning: Found unknown categories in columns [7] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
c:\Users\sijia\miniconda3\envs\dsi_participant\lib\site-packages\sklearn\preprocessing\_encoders.py:242: UserWarning: Found unknown categories in columns [7] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


{'fit_time': array([32.53464055, 30.88096499, 34.17123127, 33.37156606, 29.28257585]),
 'score_time': array([0.5648818 , 0.47909093, 0.63701916, 0.45418501, 0.4244523 ]),
 'test_neg_log_loss': array([-0.35995281, -0.39050292, -0.36224194, -0.35225141, -0.3669761 ]),
 'train_neg_log_loss': array([-0.08021765, -0.080083  , -0.08070092, -0.08080909, -0.08171859]),
 'test_roc_auc': array([0.90155819, 0.89693883, 0.90370504, 0.90296054, 0.90640886]),
 'train_roc_auc': array([1.        , 0.99999999, 0.99999983, 1.        , 1.        ]),
 'test_accuracy': array([0.8565945 , 0.84628378, 0.85288698, 0.85810811, 0.85519042]),
 'train_accuracy': array([0.99996161, 0.99992322, 0.99992322, 1.        , 0.99996161]),
 'test_balanced_accuracy': array([0.77414693, 0.76572407, 0.77965355, 0.77590648, 0.77942858]),
 'train_balanced_accuracy': array([0.99992028, 0.99984059, 0.99994943, 1.        , 0.99992029])}

Display the fold-level results as a pandas data frame and sorted by negative log loss of the test (validation) set.

In [26]:
# Create a DataFrame to display fold-level results and sort by negative log loss of the test set
cv_results_df = pd.DataFrame(cv_results).sort_values(by='test_neg_log_loss', ascending=False)

# Display the results using standard print or Jupyter display
print(cv_results_df)  # or use display() if you're using a Jupyter notebook
cv_results_df

    fit_time  score_time  test_neg_log_loss  train_neg_log_loss  test_roc_auc  \
3  33.371566    0.454185          -0.352251           -0.080809      0.902961   
0  32.534641    0.564882          -0.359953           -0.080218      0.901558   
2  34.171231    0.637019          -0.362242           -0.080701      0.903705   
4  29.282576    0.424452          -0.366976           -0.081719      0.906409   
1  30.880965    0.479091          -0.390503           -0.080083      0.896939   

   train_roc_auc  test_accuracy  train_accuracy  test_balanced_accuracy  \
3            1.0       0.858108        1.000000                0.775906   
0            1.0       0.856595        0.999962                0.774147   
2            1.0       0.852887        0.999923                0.779654   
4            1.0       0.855190        0.999962                0.779429   
1            1.0       0.846284        0.999923                0.765724   

   train_balanced_accuracy  
3                 1.000000  
0   

,fit_time,score_time,test_neg_log_loss,train_neg_log_loss,test_roc_auc,train_roc_auc,test_accuracy,train_accuracy,test_balanced_accuracy,train_balanced_accuracy
3,33.371566,0.454185,-0.352251,-0.080809,0.902961,1.0,0.858108,1.000000,0.775906,1.000000
0,32.534641,0.564882,-0.359953,-0.080218,0.901558,1.0,0.856595,0.999962,0.774147,0.999920
2,34.171231,0.637019,-0.362242,-0.080701,0.903705,1.0,0.852887,0.999923,0.779654,0.999949
4,29.282576,0.424452,-0.366976,-0.081719,0.906409,1.0,0.855190,0.999962,0.779429,0.999920
1,30.880965,0.479091,-0.390503,-0.080083,0.896939,1.0,0.846284,0.999923,0.765724,0.999841


Calculate the mean of each metric. 

In [ ]:
# Calculate the mean of each metric
mean_results = cv_results_df.mean()
mean_results

fit_time                   32.048196
score_time                  0.511926
test_neg_log_loss          -0.366385
train_neg_log_loss         -0.080706
test_roc_auc                0.902314
train_roc_auc               1.000000
test_accuracy               0.853813
train_accuracy              0.999954
test_balanced_accuracy      0.774972
train_balanced_accuracy     0.999926
dtype: float64

Calculate the same performance metrics (negative log loss, ROC AUC, accuracy, and balanced accuracy) using the testing data `X_test` and `Y_test`. Display results as a dictionary.

*Tip*: both, `roc_auc()` and `neg_log_loss()` will require prediction scores from `pipe.predict_proba()`. However, for `roc_auc()` you should only pass the last column `Y_pred_proba[:, 1]`. Use `Y_pred_proba` with `neg_log_loss()`.

In [ ]:
# Evaluate the model on the test set
model_pipeline.fit(X_train, Y_train)  # Train the model
Y_pred_proba = model_pipeline.predict_proba(X_test)  # Get prediction probabilities
Y_pred = model_pipeline.predict(X_test)  # Get predictions

# Calculate performance metrics on the test set
test_metrics = {
    'neg_log_loss': log_loss(Y_test, Y_pred_proba),
    'roc_auc': roc_auc_score(Y_test, Y_pred_proba[:, 1]),
    'accuracy': accuracy_score(Y_test, Y_pred),
    'balanced_accuracy': balanced_accuracy_score(Y_test, Y_pred)
}
test_metrics

{'neg_log_loss': 0.3778568476342141,
 'roc_auc': 0.9004103850994182,
 'accuracy': 0.8544375063977889,
 'balanced_accuracy': 0.7739486472276471}

# Target Recoding

In the first code chunk of this document, we loaded the data and immediately recoded the target variable `income`. Why is this [convenient](https://scikit-learn.org/stable/modules/model_evaluation.html#binary-case)?

The specific line was:

```
adult_dt = (pd.read_csv('../05_src/data/adult/adult.data', header = None, names = columns)
              .assign(income = lambda x: (x.income.str.strip() == '>50K')*1))
```

> **Why Recoding the Target Variable is Convenient?** 
> Recoding the target variable `income` as a binary variable (`0` and `1`) immediately after loading the data is convenient for several reasons, particularly when dealing with binary classification problems in machine learning:
> 1. **Compatibility with Machine Learning Algorithms:**
    > + Most machine learning algorithms in `scikit-learn` (e.g., logistic regression, decision trees, random forests, etc.) require the target variable to be in a numeric format. By converting `'>50K'` to `1` and everything else to `0`, you prepare the data in a format that is compatible with these algorithms right from the start.
    > + This eliminates the need for additional data transformation steps before fitting your model.
> 2. **Simplified Evaluation:**
    > + Many performance metrics, like accuracy, log loss, and ROC AUC, work directly with binary numeric labels (`0` and `1`). By converting the target to a numeric binary format immediately, you make it easier to compute and interpret these metrics during model evaluation.
    > + Using binary values ensures that functions like `cross_validate` and metrics such as `roc_auc_score` and `log_loss` work seamlessly.
> 3. **Consistency and Reproducibility:**
    > + Recoding the target variable at the beginning of the data preprocessing pipeline ensures consistency throughout your workflow. Anyone working with the dataset will use the same binary coding, making the analysis more reproducible and easier to understand.
    > + This way, you maintain a clear and straightforward approach to handling the target variable.

>**Summary:** 
> Recoding the target variable as a binary variable (0 and 1) right after loading the data is convenient because it prepares the data in a machine-learning-friendly format, simplifies model evaluation, and ensures a consistent, reproducible workflow.

## Criteria

The [rubric](./assignment_2_rubric_clean.xlsx) contains the criteria for assessment.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-2`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_2.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.

# Reference

Becker,Barry and Kohavi,Ronny. (1996). Adult. UCI Machine Learning Repository. https://doi.org/10.24432/C5XW20.